In [1]:
# Import the packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import polars as pl
import json
import plotly.express as px
import us
from bs4 import BeautifulSoup 
from lxml import etree 
import threading
import requests 
from cse251 import Log
log = Log(show_terminal=False, filename_log='scraperLog.txt')#, disable_file=True)

In [6]:
# Go through all the state abbreviations (+ washington dc) and request all the state pages and put in a list along with what state they're in
stateSoups = [
    (state, BeautifulSoup(requests.get(f'https://local.churchofjesuschrist.org/en/us/{state}').content, 'html.parser'))
    # for state in ['id']
    for state in list(map(lambda st: st.abbr.lower(), us.states.STATES)) + ['dc/washington']
]

In [7]:
# Go through all the state soups, get the links on the page, and make a big dict of {'state': ['county', 'links']}
# countySoups = {}
import ezregex as er
import re

buildingData = []
def addData(state, soup):
    _countyCount = 0
    _stateCount = 0
    i = 0
    tree = etree.HTML(str(soup))

    while True:
        _countyCount
        i += 1
        try:
            # An error here means there's no more county links on the state page
            elementData = tree.xpath(f'/html/body/main/div/section/div/ul/li[{i}]/a')[0].attrib
        except IndexError as err:
            break
    
        # If it links to a page with a single church building, it looks different than one which has multiple church buildings
        # Remeber, 10 starts with a 1
        if elementData['data-count'][1] == '1' and elementData['data-count'][2] == ')':
            subSoup = BeautifulSoup(requests.get(f'https://local.churchofjesuschrist.org/{elementData["href"][6:]}').content, 'html.parser')
            subTree = etree.HTML(str(subSoup))
            county = re.search((state + '/' + er.group(er.matchMax(er.either(er.word, '-')))).str(), elementData['href']).groups()[0],
            try:
                # An error here means... there's a problem? It means we can't get the address for some reason
                address = subTree.xpath('/html/body/main/div/section[1]/div[2]/div/div/div[1]/div[1]/address/div[1]/span')[0].text
            except IndexError:
                log.write(f'IndexError at state: {state}, county: {county}, link: https://local.churchofjesuschrist.org/{elementData["href"][6:]}, stateCount: {_stateCount}, countyCount: {_countyCount}')
                continue
            # address2 = tree.xpath('/html/body/main/div/section[1]/div[2]/div/div/div[1]/div[1]/address/div[2]')[0]
            buildingData.append({
                'state': state,
                'county': county,
                'address': address,
            })
            _countyCount += 1
            _stateCount += 1
        else:
            subSoup = BeautifulSoup(requests.get(f'https://local.churchofjesuschrist.org/{elementData["href"][6:]}').content, 'html.parser')
            subTree = etree.HTML(str(subSoup))
            # Iterate through the addresses listed on the page
            k = 0
            county = re.search((state + '/' + er.group(er.word)).str(), elementData['href']).groups()[0]
            while True:
                k += 1
                try:
                    address = subTree.xpath(f'/html/body/main/div/section/div/ul/li[{k}]/article/div[2]/address/div[1]/span')[0].text
                    # address2 = subTree.xpath(f'/html/body/main/div/section/div/ul/li[{k}]/article/div[2]/address/div[2]')[0].text
                    buildingData.append({
                        'state': state,
                        'county': county,
                        'address': address,
                    })
                    _countyCount += 1
                    _stateCount += 1
                except IndexError:
                    break
        log.write(f'Done with {county} county in {state}: found {_countyCount} church building[s]')
    log.write(f'Done with state {state}: found {_stateCount} church buildings')

threads = [threading.Thread(target=addData, args=args) for args in stateSoups]
[i.start() for i in threads]
[i.join() for i in threads]
pass


In [14]:
# Don't know why it started doing this all of the sudden
buildingDataFixed = []
for i in buildingData:
    if type(i['county']) is tuple:
        i['county'] = i['county'][0]
    buildingDataFixed.append(i)


# 5859- 6529

In [3]:
addr = pl.read_csv('Attempt3-6509_addresses.csv')
addr

state,county,address
str,str,str
"""az""","""ajo""","""801 N Cedar St…"
"""al""","""alabaster""","""210 Lacey Aven…"
"""ar""","""alma""","""201 Canterbury…"
"""co""","""alamosa""","""300 Richardson…"
"""co""","""alamosa""","""927 Weber Driv…"
"""ak""","""anchorage""","""2501 Maplewood…"
"""ak""","""anchorage""","""3250 Strawberr…"
"""ak""","""anchorage""","""13111 Brayton …"
"""ak""","""anchorage""","""2240 Baxter Ro…"


In [22]:
addr = pl.DataFrame(buildingDataFixed)
addr.write_csv('Attempt3-6509_addresses.csv')
addr

state,county,address
str,str,str
"""az""","""ajo""","""801 N Cedar St…"
"""al""","""alabaster""","""210 Lacey Aven…"
"""ar""","""alma""","""201 Canterbury…"
"""co""","""alamosa""","""300 Richardson…"
"""co""","""alamosa""","""927 Weber Driv…"
"""ak""","""anchorage""","""2501 Maplewood…"
"""ak""","""anchorage""","""3250 Strawberr…"
"""ak""","""anchorage""","""13111 Brayton …"
"""ak""","""anchorage""","""2240 Baxter Ro…"


In [19]:
with open('quicksave.json', 'w') as f:
    json.dump(buildingDataFixed, f)

In [19]:
missing = dict(zip(['lat',
    'lon',
    'full_address',
    'address_line_1',
    'address_line_2',
    'area',
    'zip',
    'zip_ext',
    'plus_code',
    'size',
    'place_id',], [None]*11))
a = {'initial_address': '522 Seventh Street SE','initial_county': 'Washington, DC', 'state': 'va', }
b = {'initial_address': '4901 16th Street Northwest','initial_county': 'Washington, DC', 'state': 'va', }
a.update(missing)
b.update(missing)
missed = [
    # 2 in Washington DC cause the webpage is different than the others
    a,b,
    # Utah is missing the other 18

]


In [6]:
now = pl.read_parquet('/home/zeke/Documents/Class/DS460/full_church_building_data-20.parquet', use_pyarrow=True)
# now = pl.read_parquet('/home/zeke/Documents/Class/DS460/Attempt3-6509_addresses.csv', use_pyarrow=True)

In [25]:
# now.vstack(pl.DataFrame(missed, schema=now.schema))
display(now.head())
pl.DataFrame(missed, schema=now.schema).head()

initial_address,initial_county,state,lat,lon,full_address,address_line_1,address_line_2,area,zip,zip_ext,plus_code,size,place_id
str,str,str,f64,f64,str,str,str,str,str,str,str,f64,str
"""210 Lacey Aven…","""alabaster""","""al""",33.223753,-86.86462,"""210 Lacey Aven…","""210 LACEY AVE""","""ALABASTER AL 3…","""ALABASTER""","""35114""",null,"""865M64FP+G5""",31.701792,"""ChIJ1eqYitQniY…"
"""801 N Cedar St…","""ajo""","""az""",32.379189,-112.866316,"""801 North Ceda…","""801 N CEDAR ST…","""AJO AZ 85321-2…","""AJO""","""85321""","""2110""","""854994HM+MF""",51.353264,"""ChIJGfLW2vNY1Y…"
"""201 Canterbury…","""alma""","""ar""",35.504507,-94.227809,"""201 Canterbury…","""201 CANTERBURY…","""ALMA AR 72921-…","""ALMA""","""72921""","""4707""","""8677GQ3C+RV""",34.9065,"""ChIJDWs5meTHy4…"
"""300 Richardson…","""alamosa""","""co""",37.469908,-105.877931,"""300 Richardson…","""300 RICHARDSON…","""ALAMOSA CO 811…","""ALAMOSA""","""81101""","""2328""","""859PF49C+XR""",43.390896,"""ChIJb5QfpfV7Fo…"
"""2501 Maplewood…","""anchorage""","""ak""",61.198109,-149.844815,"""2501 Maplewood…","""2501 MAPLEWOOD…","""ANCHORAGE AK 9…","""ANCHORAGE""","""99508""","""4056""","""93HG55X4+63""",45.65418,"""ChIJ7X4z07KXyF…"


initial_address,initial_county,state,lat,lon,full_address,address_line_1,address_line_2,area,zip,zip_ext,plus_code,size,place_id
str,str,str,f64,f64,str,str,str,str,str,str,str,f64,str
"""522 Seventh St…","""Washington, DC…","""va""",null,null,null,null,null,null,null,null,null,null,null
"""4901 16th Stre…","""Washington, DC…","""va""",null,null,null,null,null,null,null,null,null,null,null


In [39]:
# soup = BeautifulSoup(requests.get('https://pe.usps.com/text/pub28/28apc_002.htm').content, 'html.parser')
with open('/home/stella/Downloads/C1 Street Suffix Abbreviations _ Postal Explorer.html', 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
# import pandas as pd
# pd.read_html('/home/stella/Documents/Class/DS460/postalTable.html')
# pd.read_html('/home/stella/Downloads/C1 Street Suffix Abbreviations _ Postal Explorer.html')

In [54]:
import io
abbr = pl.DataFrame(pd.read_html(io.StringIO(str(soup.find(id='ep533076'))))[0])
abbr.write_csv('postalTable.csv')

In [61]:
abbrs = {}
for i in abbr.select('0', '2').to_dicts():
    abbrs[i['0']] = i['2']
    

In [157]:
dumpedData = {}
apiData = [None]*8
niceData = []

In [162]:
def makeRequest(a, b, c):
    global dumpedData, niceData
    resp = requests.post(f'https://addressvalidation.googleapis.com/v1:validateAddress?key={gmapsKey}', json={
        "address": {
            "addressLines": [c],
            "locality": b,
            "administrativeArea": a,
        },
        "previousResponseId": '',
        "enableUspsCass": False,
    }).json()
    dumpedData[f'{c}, {b}, {a}'] = resp

    try: fullAddress = resp['result']['address']['formattedAddress']                                      
    except: fullAddress       = None
    try: addressFirstLine = resp['result']['uspsData']['standardizedAddress']['firstAddressLine']         
    except: addressFirstLine  = None
    try: addressSecondLine = resp['result']['uspsData']['standardizedAddress']['cityStateZipAddressLine'] 
    except: addressSecondLine = None
    try: area = resp['result']['uspsData']['standardizedAddress']['city']                                 
    except: area              = None
    try: _zip = resp['result']['uspsData']['standardizedAddress']['zipCode']                               
    except: _zip               = None
    try: zipExt = resp['result']['uspsData']['standardizedAddress']['zipCodeExtension']                   
    except: zipExt            = None
    try: lat = resp['result']['geocode']['location']['latitude']                                          
    except: lat               = None
    try: lon = resp['result']['geocode']['location']['longitude']                                         
    except: lon               = None
    try: plusCode = resp['result']['geocode']['plusCode']['globalCode']                                   
    except: plusCode          = None
    try: size = resp['result']['geocode']['featureSizeMeters']                                            
    except: size              = None
    try: placeID = resp['result']['geocode']['placeId']                                                   
    except: placeID           = None
    niceData.append((c, b, a, lat, lon, fullAddress, addressFirstLine, addressSecondLine, area, _zip, zipExt, plusCode, size, placeID))


for chunk in addr.iter_slices(200):
    threads = [threading.Thread(target=makeRequest, args=args) for args in chunk.transpose()]
    [i.start() for i in threads]
    [i.join() for i in threads]


print('Done!')
# i=0
# apiData[i] = list(addr.iter_slices(1000))[i].map_rows(makeRequest)
# apiData

Done!

In [165]:
data = pl.DataFrame(niceData)#.write_csv('quicksave-fromGoogleApi.csv')

In [167]:
data = data.rename({
    'column_0':  'initial_address',
    'column_1':  'initial_county',
    'column_2':  'state',
    'column_3':  'lat',
    'column_4':  'lon',
    'column_5':  'full_address',
    'column_6':  'address_line_1',
    'column_7':  'address_line_2',
    'column_8':  'area',
    'column_9':  'zip',
    'column_10': 'zip_ext',
    'column_11': 'plus_code',
    'column_12': 'size',
    'column_13': 'place_id',
})

In [169]:
data.write_parquet('full_church_building_data-20.parquet')

In [154]:
# for c in __builtin__.zip():
#     print(c)
#     break
for i in addr.transpose():
    print(i[0])
    break

az

In [139]:
# zip([1, 2], [3,4], [5,6])



<class 'zip'>

In [123]:
len(dumpedData)

91

In [88]:
# resp = requests.post('https://addressvalidation.googleapis.com/v1:validateAddress', json={
resp = requests.post(f'https://addressvalidation.googleapis.com/v1:validateAddress?key={gmapsKey}', json={
    # "address": '801 N Cedar St, ajo, az',
    "address": {
        "addressLines": ["801 N Cedar St"],
        "locality": "Ajo",
        "administrativeArea": "AZ",
    },
    "previousResponseId": '',
    "enableUspsCass": False,
})

In [89]:
from rich import print
print(resp.json())

{
    'result': {
        'verdict': {
            'inputGranularity': 'PREMISE',
            'validationGranularity': 'PREMISE',
            'geocodeGranularity': 'PREMISE',
            'addressComplete': True,
            'hasInferredComponents': True
        },
        'address': {
            'formattedAddress': '801 North Cedar Street, Ajo, AZ 85321-2110, USA',
            'postalAddress': {
                'regionCode': 'US',
                'languageCode': 'en',
                'postalCode': '85321-2110',
                'administrativeArea': 'AZ',
                'locality': 'Ajo',
                'addressLines': ['801 N Cedar St']
            },
            'addressComponents': [
                {
                    'componentName': {'text': '801'},
                    'componentType': 'street_number',
                    'confirmationLevel': 'CONFIRMED'
                },
                {
                    'componentName': {'text': 'North Cedar Street', 'languageCode': 'en'},
                    'componentType': 'route',
                    'confirmationLevel': 'CONFIRMED'
                },
                {
                    'componentName': {'text': 'Ajo', 'languageCode': 'en'},
                    'componentType': 'locality',
                    'confirmationLevel': 'CONFIRMED'
                },
                {
                    'componentName': {'text': 'AZ', 'languageCode': 'en'},
                    'componentType': 'administrative_area_level_1',
                    'confirmationLevel': 'CONFIRMED'
                },
                {
                    'componentName': {'text': 'USA', 'languageCode': 'en'},
                    'componentType': 'country',
                    'confirmationLevel': 'CONFIRMED',
                    'inferred': True
                },
                {
                    'componentName': {'text': '85321'},
                    'componentType': 'postal_code',
                    'confirmationLevel': 'CONFIRMED',
                    'inferred': True
                },
                {
                    'componentName': {'text': '2110'},
                    'componentType': 'postal_code_suffix',
                    'confirmationLevel': 'CONFIRMED',
                    'inferred': True
                }
            ]
        },
        'geocode': {
            'location': {'latitude': 32.3791887, 'longitude': -112.8663162},
            'plusCode': {'globalCode': '854994HM+MF'},
            'bounds': {
                'low': {'latitude': 32.379065, 'longitude': -112.8664757},
                'high': {'latitude': 32.3795649, 'longitude': -112.865999}
            },
            'featureSizeMeters': 51.353264,
            'placeId': 'ChIJGfLW2vNY1YARCdAW5-jF4c4',
            'placeTypes': ['premise']
        },
        'metadata': {'business': True, 'poBox': False},
        'uspsData': {
            'standardizedAddress': {
                'firstAddressLine': '801 N CEDAR ST',
                'cityStateZipAddressLine': 'AJO AZ 85321-2110',
                'city': 'AJO',
                'state': 'AZ',
                'zipCode': '85321',
                'zipCodeExtension': '2110'
            },
            'deliveryPointCode': '01',
            'deliveryPointCheckDigit': '6',
            'dpvConfirmation': 'Y',
            'dpvFootnote': 'AABB',
            'dpvCmra': 'N',
            'dpvVacant': 'N',
            'dpvNoStat': 'Y',
            'carrierRoute': 'C001',
            'carrierRouteIndicator': 'C',
            'postOfficeCity': 'AJO',
            'postOfficeState': 'AZ',
            'fipsCountyCode': '019',
            'county': 'PIMA',
            'elotNumber': '0227',
            'elotFlag': 'A',
            'addressRecordType': 'S',
            'dpvNoStatReasonCode': 2,
            'dpvDrop': 'N',
            'dpvThrowback': 'N',
            'dpvNonDeliveryDays': 'N',
            'dpvNoSecureLocation': 'N',
            'dpvPbsa': 'N',
            'dpvDoorNotAccess

In [105]:
fullAddress = resp.json()['result']['address']['formattedAddress']
addressFirstLine = resp.json()['result']['uspsData']['standardizedAddress']['firstAddressLine']
addressSecondLine = resp.json()['result']['uspsData']['standardizedAddress']['cityStateZipAddressLine']
area = resp.json()['result']['uspsData']['standardizedAddress']['city']
zip = resp.json()['result']['uspsData']['standardizedAddress']['zipCode']
zipExt = resp.json()['result']['uspsData']['standardizedAddress']['zipCodeExtension']
lat = resp.json()['result']['geocode']['location']['latitude']
lon = resp.json()['result']['geocode']['location']['longitude']
plusCode = resp.json()['result']['geocode']['plusCode']['globalCode']
size = resp.json()['result']['geocode']['featureSizeMeters']
placeID = resp.json()['result']['geocode']['placeId']

In [104]:
resp.json()['result']['uspsData']


{
    'standardizedAddress': {
        'firstAddressLine': '801 N CEDAR ST',
        'cityStateZipAddressLine': 'AJO AZ 85321-2110',
        'city': 'AJO',
        'state': 'AZ',
        'zipCode': '85321',
        'zipCodeExtension': '2110'
    },
    'deliveryPointCode': '01',
    'deliveryPointCheckDigit': '6',
    'dpvConfirmation': 'Y',
    'dpvFootnote': 'AABB',
    'dpvCmra': 'N',
    'dpvVacant': 'N',
    'dpvNoStat': 'Y',
    'carrierRoute': 'C001',
    'carrierRouteIndicator': 'C',
    'postOfficeCity': 'AJO',
    'postOfficeState': 'AZ',
    'fipsCountyCode': '019',
    'county': 'PIMA',
    'elotNumber': '0227',
    'elotFlag': 'A',
    'addressRecordType': 'S',
    'dpvNoStatReasonCode': 2,
    'dpvDrop': 'N',
    'dpvThrowback': 'N',
    'dpvNonDeliveryDays': 'N',
    'dpvNoSecureLocation': 'N',
    'dpvPbsa': 'N',
    'dpvDoorNotAccessible': 'N',
    'dpvEnhancedDeliveryCode': 'Y'
}

In [67]:
from Cope import catFile
gmapsKey = catFile('/home/stella/Documents/Keys/GoogleMapsTargetCreationProject')

In [120]:
etree.HTML(str(stateSoups[0][1])).xpath(f'/html/body/main/div/section/div/ul/li[{4}]/a')[0].attrib

{'class': 'Directory-listLink', 'data-count': '(1)', 'data-ya-track': 'todirectory', 'href': '../../en/us/al/anniston/1217-lenlock-lane'}

In [ ]:
# stateSoups

In [98]:
import ezregex as er
s = '../../en/us/al/wetumpka/1405-chapel-road'
r = 'al/' + er.group(er.word)
# r = er.line_ends_with(er.word + '/' + er.matchMax(er.word + er.optional('-')))
r.test(s)
import re
re.search(r.str(), s).groups()[0]

╭───────────────────────────────────────────────── Testing Regex ─────────────────────────────────────────────────╮
│ Testing expression (from ["/tmp/ipykernel_39851/4245750080.py", line 5]):                                       │
│         al/(\w+)                                                                                                │
│ for matches in:                                                                                                 │
│         ../../en/us/al/wetumpka/1405-chapel-road                                                                │
│                                                                                                                 │
│ Match = "al/wetumpka" (12:23)                                                                                   │
│ Unnamed Groups:                                                                                                 │
│         1: "wetumpka" (15:23)                                                                                   │
│                                                                                                                 │
╰──────────────────────────────────────────────────── Found  ─────────────────────────────────────────────────────╯

'wetumpka'

In [ ]:
# Go through all the links, and fetch their pages
countySoups = {}
for state, links in countyLinks:
    countySoups['state'] = [
        BeautifulSoup(requests.get(f'https://local.churchofjesuschrist.org/{link[6:]}').content, 'html.parser')
        for link in links
    ]

In [70]:
for soup in soups:
    dom = etree.HTML(str(soups[0])) 
    e = dom.xpath('/html/body/main/div/section/div/ul/li[1]/a')[0]
    display(e.attrib['href'])

'../../en/us/al/alabaster/210-lacey-avenue'

In [76]:
'../../en/us/al/alabaster/210-lacey-avenue'[6:]

'en/us/al/alabaster/210-lacey-avenue'

In [50]:
'https://local.churchofjesuschrist.org/en/us/'

{'class': 'Directory-listItem'}

In [61]:
import us



[
    'al',
    'ak',
    'az',
    'ar',
    'ca',
    'co',
    'ct',
    'de',
    'fl',
    'ga',
    'hi',
    'id',
    'il',
    'in',
    'ia',
    'ks',
    'ky',
    'la',
    'me',
    'md',
    'ma',
    'mi',
    'mn',
    'ms',
    'mo',
    'mt',
    'ne',
    'nv',
    'nh',
    'nj',
    'nm',
    'ny',
    'nc',
    'nd',
    'oh',
    'ok',
    'or',
    'pa',
    'ri',
    'sc',
    'sd',
    'tn',
    'tx',
    'ut',
    'vt',
    'va',
    'wa',
    'wv',
    'wi',
    'wy'
]

In [ ]:



URL = "https://en.wikipedia.org/wiki/Nike,_Inc."

HEADERS = ({'User-Agent': 
			'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \ 
			(KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',\ 
			'Accept-Language': 'en-US, en;q=0.5'}) 

webpage = requests.get(URL, headers=HEADERS) 
soup = BeautifulSoup(webpage.content, "html.parser") 
dom = etree.HTML(str(soup)) 
print(dom.xpath('//*[@id="firstHeading"]')[0].text) 
